In [ ]:
#import packages
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn import metrics
from sklearn.linear_model import RidgeClassifier, LogisticRegression
from sklearn.metrics import accuracy_score
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [ ]:
#mounting google drive on colab notebook
from google.colab import drive
drive.mount('/d')


Drive already mounted at /d; to attempt to forcibly remount, call drive.mount("/d", force_remount=True).


In [ ]:
#reading normal traffic data from csv file
data1 = pd.read_csv("/d/My Drive/Master's project code_Arun/2010/2010/Fomatted CSV Files/cisc_normalTraffic_train_formatted.csv")

In [ ]:
#reading anomalous traffic data from csv file
data2 = pd.read_csv("/d/My Drive/Master's project code_Arun/2010/2010/Fomatted CSV Files/cisc_anomalous_Traffic_test_formatted.csv")

In [ ]:
#extact only 4 columns from both tables to a new dataframe
X_data1 = data1.loc[:5000,['GET http','POST http','POST Body']]
y_data1 = data1.loc[:5000,['class']]
X_data2 = data2.loc[:5000,['GET http','POST http','POST Body']]
y_data2 = data2.loc[:5000,['class']]

In [ ]:
#concat the data from the two tables
X = pd.concat([X_data1, X_data2], ignore_index=True)
y = pd.concat([y_data1, y_data2], ignore_index=True)

#checking the number of classes in the data
class_set = set(y['class'].tolist())
print(class_set)

#convert the class labels to integers
label_encoder = LabelEncoder()
y = np.array(label_encoder.fit_transform(y['class']))

{' Valid', ' Attack'}


In [ ]:
#fill the nans with 0
X.fillna(0, inplace = True)

#iterate through each row and remove redundant chunks of text from the 3 columns
for i,rows in X.iterrows():
    l = []
    for j in rows:
        if j!=0:
            if j.startswith('//localhost:8080/tienda1/'):
                j = j[len('//localhost:8080/tienda1/') :]
            if j.endswith(' HTTP/1.1'):
                j = j[:len(j)-len(' HTTP/1.1')]
            l.append(j)
        else:
            l.append(str(j))
   
    X.replace(to_replace = rows[0], value = l[0], inplace = True) 
    X.replace(to_replace = rows[1], value = l[1], inplace = True) 
    X.replace(to_replace = rows[2], value = l[2], inplace = True)
    

In [ ]:
X.head(10)

,GET http,POST http,POST Body
0,imagenes/nuestratierra.jpg,0,0
1,publico/registro.jsp?modo=registro&login=sikan...,0,0
2,miembros/imagenes/zarauz.jpg,0,0
3,publico/autenticar.jsp?modo=entrar&login=chuan...,0,0
4,publico/vaciar.jsp?B2=Vaciar+carrito,0,0
5,miembros/imagenes/zarauz.jpg,0,0
6,0,publico/autenticar.jsp,nanmodo=entrar&login=janna&pwd=eusquero&rememb...
7,0,publico/caracteristicas.jsp,nanid=1
8,miembros/index.jsp,0,0
9,imagenes/logo.gif,0,0


In [ ]:
# #the data in both the columns are decoded
# from urllib.parse import unquote
# for i,rows in X.iterrows():
#     if rows[0] != 0 :
#         rows[0] = unquote(rows[0])
#     if rows[1] != 0:
#         rows[1] = unquote(rows[1])
#     if rows[2] != 0:
#         rows[2] = unquote(rows[2])
  

In [ ]:
#Bag of Words
vectorizer = CountVectorizer()  
vectorizer1 = CountVectorizer()
vectorizer2 = CountVectorizer()

In [ ]:
#BOW for GET http column
vectorizer.fit(X['GET http'])
vector = vectorizer.transform(X['GET http']).toarray()

In [ ]:
#BOW for POST http column
vectorizer1.fit(X['POST http'])
vector1 = vectorizer1.transform(X['POST http']).toarray()

In [ ]:
#BOW for POST Body column
vectorizer2.fit(X['POST Body'])
vector2 = vectorizer2.transform(X['POST Body']).toarray()

In [ ]:
#combining the BOW outputs of the 3 columns
X_train = np.concatenate((vector,vector1,vector2),axis = 1)
y_train = np.reshape(y,(10002,))

In [ ]:
#checking shape of the arrays before training
print(X_train.shape)
print(y_train.shape)

(10002, 21285)
(10002,)


In [ ]:
#splitting data into train and test data
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.20)

In [ ]:
 from sklearn.neural_network import MLPClassifier
 model = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(4, 2), random_state=1)
 model.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(4, 2), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=1, shuffle=True, solver='lbfgs',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [ ]:
r = model.predict(X_test)
accuracy = accuracy_score(y_true = y_test, y_pred = r)

In [ ]:
print("MLP Classifier :")
print("Accuracy :", accuracy)
print("Precision Score :", precision_score(y_test, r, average = 'weighted'))
print("Recall score :", recall_score(y_test, r, average = 'weighted'))
print("F1 - score :", f1_score(y_test, r, average = 'weighted'))
print("Confusion Matrix :\n",confusion_matrix(y_test, r))
print("Classification Report :\n",classification_report(y_test, r, target_names=class_set))

MLP Classifier :
Accuracy : 0.815592203898051
Precision Score : 0.8624749363978327
Recall score : 0.815592203898051
F1 - score : 0.809695100574014
Confusion Matrix :
 [[989   5]
 [364 643]]
Classification Report :
               precision    recall  f1-score   support

       Valid       0.73      0.99      0.84       994
      Attack       0.99      0.64      0.78      1007

    accuracy                           0.82      2001
   macro avg       0.86      0.82      0.81      2001
weighted avg       0.86      0.82      0.81      2001



In [ ]:
#Model 1: Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
r = model.predict(X_test)
accuracy = accuracy_score(y_true = y_test, y_pred = r)

In [ ]:
print("Random Forest Classifier :")
print("Accuracy :", accuracy)
print("Precision Score :", precision_score(y_test, r, average = 'weighted'))
print("Recall score :", recall_score(y_test, r, average = 'weighted'))
print("F1 - score :", f1_score(y_test, r, average = 'weighted'))
print("Confusion Matrix :\n",confusion_matrix(y_test, r))
print("Classification Report :\n",classification_report(y_test, r, target_names=class_set))

Random Forest Classifier :
Accuracy : 0.9205397301349325
Precision Score : 0.9222451295398434
Recall score : 0.9205397301349325
F1 - score : 0.9204771786448604
Confusion Matrix :
 [[946  48]
 [111 896]]
Classification Report :
               precision    recall  f1-score   support

       Valid       0.89      0.95      0.92       994
      Attack       0.95      0.89      0.92      1007

    accuracy                           0.92      2001
   macro avg       0.92      0.92      0.92      2001
weighted avg       0.92      0.92      0.92      2001



In [ ]:
#model 2 : Logistic Regression
model = LogisticRegression()
model.fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
r = model.predict(X_test)
accuracy = accuracy_score(y_true = y_test, y_pred = r)

In [ ]:
print("Logistic Regression Classifier :")
print("Accuracy :", accuracy)
print("Precision Score :", precision_score(y_test, r, average = 'weighted'))
print("Recall score :", recall_score(y_test, r, average = 'weighted'))
print("F1 - score :", f1_score(y_test, r, average = 'weighted'))
print("Confusion Matrix :\n",confusion_matrix(y_test, r))
print("Classification Report :\n",classification_report(y_test, r, target_names=class_set))

Logistic Regression Classifier :
Accuracy : 0.9435282358820589
Precision Score : 0.9441892340046228
Recall score : 0.9435282358820589
F1 - score : 0.9434996139783035
Confusion Matrix :
 [[918  76]
 [ 37 970]]
Classification Report :
               precision    recall  f1-score   support

       Valid       0.96      0.92      0.94       994
      Attack       0.93      0.96      0.94      1007

    accuracy                           0.94      2001
   macro avg       0.94      0.94      0.94      2001
weighted avg       0.94      0.94      0.94      2001



In [ ]:
#Model 3: Stochastic Gradient Descent Classifier
from sklearn.linear_model import SGDClassifier
model = SGDClassifier()
model.fit(X_train, y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
r = model.predict(X_test)
accuracy = accuracy_score(y_true = y_test, y_pred = r)
print(accuracy)

0.952023988005997


In [ ]:
print("Stochastic Gradient Descent Classifier :")
print("Accuracy :", accuracy)
print("Precision Score :", precision_score(y_test, r, average = 'weighted'))
print("Recall score :", recall_score(y_test, r, average = 'weighted'))
print("F1 - score :", f1_score(y_test, r, average = 'weighted'))
print("Confusion Matrix :\n",confusion_matrix(y_test, r))
print("Classification Report :\n",classification_report(y_test, r, target_names=class_set))

Stochastic Gradient Descent Classifier :
Accuracy : 0.952023988005997
Precision Score : 0.9521985087124698
Recall score : 0.952023988005997
F1 - score : 0.9520160777227042
Confusion Matrix :
 [[936  58]
 [ 38 969]]
Classification Report :
               precision    recall  f1-score   support

       Valid       0.96      0.94      0.95       994
      Attack       0.94      0.96      0.95      1007

    accuracy                           0.95      2001
   macro avg       0.95      0.95      0.95      2001
weighted avg       0.95      0.95      0.95      2001



In [ ]:
#Model4: Decision Tree Classifier
from sklearn import tree
model = tree.DecisionTreeClassifier()
model.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
r = model.predict(X_test)
accuracy = accuracy_score(y_true = y_test, y_pred = r)
print(accuracy)

0.9270364817591205


In [ ]:
print("Decision Tree Classifier :")
print("Accuracy :", accuracy)
print("Precision Score :", precision_score(y_test, r, average = 'weighted'))
print("Recall score :", recall_score(y_test, r, average = 'weighted'))
print("F1 - score :", f1_score(y_test, r, average = 'weighted'))
print("Confusion Matrix :\n",confusion_matrix(y_test, r))
print("Classification Report :\n",classification_report(y_test, r, target_names=class_set))

Decision Tree Classifier :
Accuracy : 0.9270364817591205
Precision Score : 0.9272710560435682
Recall score : 0.9270364817591205
F1 - score : 0.9270202945092814
Confusion Matrix :
 [[909  85]
 [ 61 946]]
Classification Report :
               precision    recall  f1-score   support

       Valid       0.94      0.91      0.93       994
      Attack       0.92      0.94      0.93      1007

    accuracy                           0.93      2001
   macro avg       0.93      0.93      0.93      2001
weighted avg       0.93      0.93      0.93      2001

